<a href="https://colab.research.google.com/github/E-tech-coder/DataScienceCapstoneProject/blob/Elena/RuleBased.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import pandas as pd

df = pd.read_csv("/content/DataScienceCapstoneProject/linkedin_experience_annotated.csv").drop(columns="linkedin").set_index("person_id").reset_index()

In [28]:
df.head(20)

,person_id,organization,position,startDate,endDate,status,department,seniority
0,0,Depot4Design GmbH,Prokurist,2019-08,NaN,ACTIVE,Other,Management
1,0,Depot4Design GmbH,CFO,2019-07,NaN,ACTIVE,Other,Management
2,0,Depot4Design GmbH,Betriebswirtin,2019-07,NaN,ACTIVE,Other,Professional
3,0,Depot4Design GmbH,Prokuristin,2019-07,NaN,ACTIVE,Other,Management
4,0,Depot4Design GmbH,CFO,2019-07,NaN,ACTIVE,Other,Management
5,0,Nagel Car Group,Buchhalterin,2000-05,2019-06,INACTIVE,Other,Professional
6,1,Computer Solutions,Solutions Architect,2024-03,NaN,ACTIVE,Information Technology,Professional
7,1,Computer Solutions,Senior Network Engineer,2019-07,2024-03,INACTIVE,Information Technology,Senior
8,1,Texas A&M University-Corpus Christi,Manager of Network Services,2017-02,2019-07,INACTIVE,Information Technology,Professional
9,1,Texas A&M University-Corpus Christi,Infrastructure Administrator II,2015-06,2017-02,INACTIVE,Information Technology,Professional


In [29]:
BOW = dict()
BOW["Marketing"] = "director"

In [30]:
BOW

{'Marketing': 'director'}

In [40]:
dpt = pd.read_csv("/content/DataScienceCapstoneProject/department-v2.csv")

In [48]:
dpt['label'].drop_duplicates()

,label
0,Marketing
1,Project Management
171,Administrative
202,Business Development
374,Consulting
476,Human Resources
487,Information Technology
2609,Other
2613,Purchasing
2636,Sales


In [59]:
BOW = dict()

for label in dpt['label'].drop_duplicates() :
  x = []
  for item in dpt[dpt["label"] == label]["text"].str.split(" ") :
    x.extend(item)
    x = list(dict.fromkeys(x))
  BOW[label] = x


In [70]:
len(BOW.get("Customer Support"))

81

In [72]:
for key in BOW.keys():
  for word in str("Infrastructure Administrator II").split(" ")

Marketing
Project Management
Administrative
Business Development
Consulting
Human Resources
Information Technology
Other
Purchasing
Sales
Customer Support


In [73]:
for word in str("Infrastructure Administrator II").split(" "):
  print(word)

Infrastructure
Administrator
II


In [ ]:
BOW.get("Project Management")

In [77]:
i = 0
for word in str("Infrastructure Administrator II").split(" "):
  if word in BOW.get("Information Technology"):
    print(word)
    i+=1
print(i)

Infrastructure
Administrator
2


In [91]:
df["WordAppearance"] = 0
df["PredictedPosition_BOW"] =""
df["position_words"] = df["position"].str.split()

In [96]:
for label, BOW_words in BOW.items():
  bow_set = set(BOW_words)
  # Count how many words of the position appear in the bag of words of the label
  for idx,words in df["position_words"].items():
    count=sum(word in bow_set for word in words)
        # Update prediction if the current word count is greater than the previous one
    if count > df.at[idx, "WordAppearance"] :
      df.at[idx, "WordAppearance"] = count
      df.at[idx, "PredictedPosition_BOW"] = label



In [98]:
df.to_csv("/content/DataScienceCapstoneProject/BOW-Position-Predict.csv")